In [1]:
# Import all the required libraries

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
# Read the text

with open('sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [3]:
# Now let’s tokenize the text to create a sequence of words:

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
# Now let’s create input-output pairs by splitting the text into sequences of tokens and forming n-grams from the sequences:

input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)          

In [5]:
# Now let’s pad the input sequences to have equal length:

max_seq_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre'))

In [6]:
# Now let’s split the sequences into input and output:

X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [7]:
# Now let’s convert the output to one-hot encode vectors:

y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

# In the above code, we are converting the output array into a suitable format for training a model,
# where each target word is represented as a binary vector.



In [8]:
# Now let’s build a neural network architecture to train the model:

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_seq_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [9]:
# Now let’s compile and train the model:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100


3010/3010 [==============================] - 161s 50ms/step - loss: 6.2502 - accuracy: 0.0758
Epoch 2/100
3010/3010 [==============================] - 142s 47ms/step - loss: 5.5163 - accuracy: 0.1239
Epoch 3/100
3010/3010 [==============================] - 142s 47ms/step - loss: 5.1310 - accuracy: 0.1469
Epoch 4/100
3010/3010 [==============================] - 141s 47ms/step - loss: 4.8047 - accuracy: 0.1660
Epoch 5/100
3010/3010 [==============================] - 134s 45ms/step - loss: 4.4987 - accuracy: 0.1836
Epoch 6/100
3010/3010 [==============================] - 137s 46ms/step - loss: 4.2096 - accuracy: 0.2041
Epoch 7/100
3010/3010 [==============================] - 140s 47ms/step - loss: 3.9323 - accuracy: 0.2291
Epoch 8/100
3010/3010 [==============================] - 141s 47ms/step - loss: 3.6725 - accuracy: 0.2578
Epoch 9/100
3010/3010 [==============================] - 142s 47ms/step - loss: 3.4275 - accuracy: 0.2911
Epoch 10/100
3010/3010 [===================

In [11]:
seed_text = "I will leave if they"
next_words = 3

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

# The above code generates the next word predictions based on a given seed text.
# The ‘seed_text’ variable holds the initial text.

1/1 [==============================] - 0s 126ms/step
I will leave if they can come with
